In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
class NaiveDense:
  def __init__(self, input_size, output_size, activation):
    self.activation = activation

    w_shape = (input_size, output_size)
    w_initial_value = tf.random.uniform(w_shape, minval = 0, maxval = 1e-1)
    self.W = tf.Variable(w_initial_value)

    b_shape = (output_size,)
    b_initial_value = tf.zeros(b_shape)
    self.b = tf.Variable(b_initial_value)


  def __call__(self, inputs):
    return self.activation(tf.matmul(inputs, self.W) + self.b)

  @property
  def weights(self):
    return [self.W, self.b]


In [3]:
class NaiveSequential:
  def __init__(self, layers):
    self.layers = layers

  def __call__(self, inputs):
    x = inputs
    for layer in self.layers:
      x = layer(x)
    return x


  @property
  def weights(self):
    weights = []
    for layer in self.layers:
      weights += layer.weights
    return weights

In [4]:
model = NaiveSequential([
    NaiveDense(input_size = 28*28, output_size = 512, activation = tf.nn.relu),
    NaiveDense(input_size = 512, output_size = 10, activation = tf.nn.softmax)
])

In [19]:
import math

class BatchGenerator:
  def __init__(self, images, labels, batch_size = 128):
    self.index = 0
    self.images = images
    self.labels = labels
    self.batch_size = batch_size
    self.num_batches = math.ceil(len(images) / batch_size)


  def next(self):
    images = self.images[self.index: self.index + self.batch_size]
    labels = self.labels[self.index: self.index + self.batch_size]
    self.index += self.batch_size
    return images, labels


In [20]:
def one_training_step(model, images_batch, labels_batch):
  with tf.GradientTape() as tape:
    predictions = model(images_batch)
    per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(labels_batch, predictions)
    average_loss = tf.reduce_mean(per_sample_losses)
  gradients = tape.gradient(average_loss, model.weights)
  update_weights(gradients, model.weights)
  return average_loss

In [21]:
learning_rate = 1e-3
def update_weights(gradients, weights):
  for g,w in zip(gradients, weights):
    w.assign_sub(g * learning_rate)

In [25]:
def fit(model, images, labels, epochs = 10, batch_size = 128):
  for epoch in range(epochs):
    print(f"Epoch: {epoch+1}")

    batch_generator = BatchGenerator(images, labels)
    for batch_counter in range(batch_generator.num_batches):
      images_batch, labels_batch = batch_generator.next()
      loss = one_training_step(model, images_batch, labels_batch)
      if batch_counter % 100 == 0:
        print(f"loss at {batch_counter}: {loss:.2f}")


In [26]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28*28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28*28))
test_images = test_images.astype("float32") / 255

In [27]:
fit(model, train_images, train_labels, epochs = 10, batch_size = 128)

Epoch: 1
loss at 0: 8.37
loss at 100: 2.19
loss at 200: 2.18
loss at 300: 2.06
loss at 400: 2.20
Epoch: 2
loss at 0: 1.87
loss at 100: 1.83
loss at 200: 1.81
loss at 300: 1.69
loss at 400: 1.82
Epoch: 3
loss at 0: 1.54
loss at 100: 1.54
loss at 200: 1.48
loss at 300: 1.40
loss at 400: 1.49
Epoch: 4
loss at 0: 1.29
loss at 100: 1.30
loss at 200: 1.22
loss at 300: 1.18
loss at 400: 1.24
Epoch: 5
loss at 0: 1.09
loss at 100: 1.13
loss at 200: 1.02
loss at 300: 1.02
loss at 400: 1.07
Epoch: 6
loss at 0: 0.95
loss at 100: 0.99
loss at 200: 0.89
loss at 300: 0.90
loss at 400: 0.95
Epoch: 7
loss at 0: 0.85
loss at 100: 0.89
loss at 200: 0.79
loss at 300: 0.81
loss at 400: 0.86
Epoch: 8
loss at 0: 0.77
loss at 100: 0.81
loss at 200: 0.71
loss at 300: 0.74
loss at 400: 0.80
Epoch: 9
loss at 0: 0.71
loss at 100: 0.75
loss at 200: 0.65
loss at 300: 0.69
loss at 400: 0.75
Epoch: 10
loss at 0: 0.66
loss at 100: 0.69
loss at 200: 0.60
loss at 300: 0.65
loss at 400: 0.71


In [29]:
import numpy as np

predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels= np.argmax(predictions, axis = 1)
matches = predicted_labels == test_labels
print(f"accuracy: {matches.mean(): .2f}")

accuracy:  0.82
